<a href="https://colab.research.google.com/github/germanjke/Deep_Learning_School_MIPT/blob/master/nlp_seminars/mt_seminar_done.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Семинар  
## Seq2Seq модели в машинном переводе

На лекции мы подробно познакомились с подходами к решению задачи машинного перевода.
Наиболее распространенными моделями последовательностей (seq2seq) являются модели кодер-декодер, которые (обычно) используют рекуррентную нейронную сеть (RNN) для кодирования исходного (входного) предложения в один вектор.  Вы можете думать о векторе контекста как об абстрактном представлении всего входного предложения. Этот вектор затем декодируется декодером, который учится выводить  предложение, генерируя его по одному слову за раз.


$h_t = \text{Encoder}(x_t, h_{t-1})$

У нас есть последовательность $X = \{x_1, x_2, ..., x_T\}$, где $x_1 = \text{<sos>;}, x_2 = \text{the}$, и так далее. Начальное состояние, $h_0$,  может быть инициализировано вектором из нулей или обучаемым.


Как только последнее слово, $x_T$, был подан на Encoder, мы  используем  информацию в  последнем скрытом состоянии, $h_T$, в зависимости от контекста вектор, т. е. $h_T $ это векторное представление всего исходного предложения.

После получения вектора всего предложения мы можем декодировать предложение уже на новом языке. На каждом шаге декодирования мы подаем правильное слово $y_t$,  дополняем это информацией о скрытом состоянии $s_{t-1}$, где  $s_t = \text{DecoderRNN}(y_t, s_{t-1})$


![alt text](https://i.stack.imgur.com/f6DQb.png)


Мы всегда используем $<sos>$ для первого входа в декодер, $y_1$, но для последующих входов, $y_{\text{from }t; 1}$, мы иногда будем использовать фактическое, основное истинное следующее слово в последовательности, $y_t$, а иногда использовать слово, предсказанное нашим декодером, $\hat{y}_{t-1}$. Использование настоящих токенов в декодере называется Teacher Forcing [можно тут посмотреть](https://machinelearningmastery.com/teacher-forcing-for-recurrent-neural-networks/)

## Датасет Multi30k

Загрузим датасет и посмотрим на него. Не забудем, что надо токенизировать текст перед отправкой в модель. Мы будем  использовать TorchText и spaCy( как токенизатор) , чтобы помочь вам выполнить всю необходимую предварительную обработку быстрее чем мы делали раньше. В данной работе вам предлагается написать модель Seq2Seq и обучить ее на Multi30k. В данном задание мы будем подавать на вход перевернутые предложения, так как авторы seq2seq считали, что это улучшает качество перевода.

In [1]:
!pip install spacy

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np

import spacy

import random
import math
import time

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator

In [3]:
seed = 42

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [6]:
! python3 -m spacy download en
! python3 -m spacy download de


spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 14.9MB 535kB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp36-none-any.whl size=14907056 sha256=11dddea7f12e75089f35ecaf412325799420557f79d10e664445b989c193226a
  Stored in directory: /tmp/pip-ephem-wheel-cache-_mvf0ax4/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [7]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)][::-1]

# немецкий язык является полем SRC, а английский в поле TRG
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [10]:
# В датасете содержится ~ 30к предложений средняя длина которых 11
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'),  fields = (SRC, TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:02<00:00, 595kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 173kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 162kB/s]


Давайте посмотрим что у нас с датасетом и сделаем словари для SRC и TGT

In [11]:
labels = ['train', 'validation', 'test']
dataloaders = [train_data, valid_data, test_data]
for d, l in zip(dataloaders, labels):
    print("Number of sentences in {} : {}".format(l, len(d.examples)))

Number of sentences in train : 29000
Number of sentences in validation : 1014
Number of sentences in test : 1000


In [12]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)
print("Number of words in source vocabulary", len(SRC.vocab))
print("Number of words in source vocabulary", len(TRG.vocab))

Number of words in source vocabulary 7855
Number of words in source vocabulary 5893


In [13]:
SRC.process(["ein klein gespenster.",])

tensor([[   2],
        [5886],
        [3833],
        [   0],
        [ 664],
        [   0],
        [   0],
        [5886],
        [3833],
        [   0],
        [ 664],
        [   0],
        [5886],
        [7098],
        [   0],
        [5886],
        [   0],
        [7098],
        [   0],
        [5886],
        [7011],
        [   4],
        [   3]])

In [14]:
SRC.vocab.itos[5]

'ein'

## BLEU

Чтобы сравнивать моделей, надо иметь численную метрику качества. Качество перевода можно оценить с помощью метрики **BLEU**. Как она работает:

Пусть у нас есть:

Предложенный перевод: `the cat mat`

Эталонный перевод: `the cat is on the mat`

In [15]:
pred = ["the", "cat", "mat"]
target = ["the", "cat", "is", "on", "the", "mat"]

Соберем н-граммы из предложенного переводы и посчитаем, сколько из них содержится в эталонном переводе.

In [16]:
# n-grams in pred
unigrams = [(i,) for i in pred]
bigrams = [(i, j) for i, j in zip(pred[:-1], pred[1:])]
trigrams = [tuple(pred)]

# n-grams in target
target_unigrams = [(i,) for i in target]
target_bigrams = [(i, j) for i, j in zip(target[:-1], target[1:])]
target_trigrams = [(i, j, k) 
                   for i, j, k in zip(target[:-2], target[1:-1], target[2:])]

In [17]:
count_unigrams = sum(uni in target_unigrams for uni in unigrams)
count_bigrams = sum(bi in target_bigrams for bi in bigrams)
count_trigrams = sum(tri in target_trigrams for tri in trigrams)

In [19]:
bigrams

[('the', 'cat'), ('cat', 'mat')]

Для BLEU нам надо поделить число посчитанных н-грамм на их количество. Если мы смотрели на разные н-граммы, то BLEU будет являться взвешенной суммой.

In [18]:
bleu_uni = count_unigrams / len(unigrams)
bleu_bi = count_bigrams / len(bigrams)
bleu_tri = count_trigrams / len(trigrams)
print(f"BLEU-uni: {bleu_uni}\nBLEU-bi: {bleu_bi}\nBLEU-tri: {bleu_tri}")

BLEU-uni: 1.0
BLEU-bi: 0.5
BLEU-tri: 0.0


In [20]:
bleu = (bleu_uni + bleu_bi + bleu_tri) / 3
assert bleu == 0.5
print(f"BLEU: {bleu}")

BLEU: 0.5


Считать вручную эту метрику не требуется. Уже есть готовые реализации, например в библиотеке `nltk`. Эта реализация использует модифицированные подсчет н-грамм и усреднение bleu для каждой н-граммы, поэтому это число будет отличаться от простой реализации. 

Заметим, что для посчета могут использоваться несколько эталонных переводов. Поэтому эталонные переводы передаются списком.

In [21]:
from nltk.translate.bleu_score import sentence_bleu

print(f"BLEU: {sentence_bleu([target], pred, weights=(1/2, 1/2))}")

BLEU: 0.2601300475114445


BLEU – не единственная метрика для перевода. Ещё есть ROGUE, METEOR, WER.

## Encoder

Напишем для начала простой Encoder, который реализует следующий функционал:

$ (h_t, c_t) = \text{LSTM}(x_t, (h_{t-1}, c_{t-1}))$

В  методе forward мы передаем исходное предложение $X$, которое преобразуется в embeddings, к которым применяется dropout . Эти вектора затем передаются в RNN. Когда мы передадим всю последовательность RNN, он автоматически выполнит для нас рекуррентный расчет скрытых состояний по всей последовательности! Вы можете заметить, что мы не передаем начальное скрытое или состояние ячейки в RNN. Это происходит потому, что, как отмечено в документации, если никакое скрытое состояние/ячейка не передается RNN, он автоматически создаст начальное скрытое состояние/ячейка как тензор всех нулей.

In [23]:
class Encoder(nn.Module):
    def __init__(self, input_size, emb_size, hidden_size, num_layers=2, dropout=0.1):
        super().__init__()
        
        self.input_size = input_size
        self.emb_size = emb_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(input_size, emb_size)
 
        self.rnn = nn.LSTM(
            emb_size, hidden_size, 
            num_layers=num_layers,
            dropout=dropout,
        ) 
        #(lstm embd, hid, layers, dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        """
        :param: src sentences (src_len x batch_size)
        """
        # embedded = <TODO> (src_len x batch_size x embd_dim)
        embedded = self.embedding(src)
        # dropout over embedding
        embedded = self.dropout(embedded)
        outputs, (hidden, cell) = self.rnn(embedded)
        # [Attention return is for lstm, but you can also use gru]
        return hidden, cell

## Decoder
Похожий на Encoder, но со слоем проекцией, который переводит из hidden_dim в output

In [24]:
class Decoder(nn.Module):
    def __init__(self, output_size, emb_size, hidden_size, num_layer=2, dropout=0.1):
        super().__init__()

        self.emb_size = emb_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.dropout = dropout
        
        self.embedding = nn.Embedding(output_size, emb_size)
        
        self.rnn = nn.LSTM(
            emb_size, hidden_size,
            num_layers=num_layers,
            dropout=self.dropout,
        )
        #(lstm embd, hid, layers, dropout)
        
        self.out = nn.Linear(hidden_size, output_size)
        # Projection :hid_dim x output_dim
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input_, hidden, cell):
        
        
        # (1x batch_size)
        input_ = input_.unsqueeze(0)
        
        # (1 x batch_size x emb_dim)
        embedded = self.embedding(input_) # embd over input and dropout 
        embedded = self.dropout(embedded)
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #sent len and n directions will always be 1 in the decoder
        
        # (batch_size x output_dim)
        
        prediction = self.out(output.squeeze(0)) #project out of the rnn on the output dim 
        
        return prediction, hidden, cell


## Seq2Seq module

In [25]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        # Hidden dimensions of encoder and decoder must be equal
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self._init_weights()  
    
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        """
        :param: src (src_len x batch_size)
        :param: tgt
        :param: teacher_forcing_ration : if 0.5 then every second token is the ground truth input
        """
        
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_size
        
        #tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src) # TODO pass src throw encoder
        
        #first input to the decoder is the <sos> tokens
        input_ = trg[0] # TODO trg[idxs]
        
        for t in range(1, max_len):
            output, hidden, cell = self.decoder(input_, hidden, cell) #TODO pass state and input throw decoder 
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            input_ = (trg[t] if teacher_force else top1)
        
        return outputs
    
    def _init_weights(self):
        p = 0.08
        for name, param in self.named_parameters():
            nn.init.uniform_(param.data, -p, p)

In [26]:
input_size = len(SRC.vocab)
output_size = len(TRG.vocab)
src_emb_size =  tgt_emb_size = 128
hidden_size = 256
num_layers =  3
dropout = 0.2

batch_size = 128
PAD_IDX = TRG.vocab.stoi['<pad>']

iterators = BucketIterator.splits((train_data, valid_data, test_data),
                                  batch_size = batch_size, device = device)
train_iterator, valid_iterator, test_iterator = iterators

encoder = Encoder(input_size, src_emb_size, hidden_size, num_layers, dropout)
decoder = Decoder(output_size, tgt_emb_size, hidden_size, num_layers, dropout)
model = Seq2Seq(encoder, decoder, device).to(device)

In [27]:
from tqdm.notebook import tqdm


def training(model, iterator, optimizer, criterion, clip): 
    model.train()
    
    epoch_loss = 0
    for i, batch in tqdm(enumerate(iterator)):
        src = batch.src
        trg = batch.trg
        
        output = model(src, trg)
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [28]:
def validating(model, iterator, criterion):
    model.eval()
    
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing !!

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)
            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
    return epoch_loss / len(iterator)

In [29]:
max_epochs = 10
CLIP = 1

# TODO
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

best_valid_loss = float('inf')

for epoch in range(max_epochs):
    
    train_loss = round(training(model, train_iterator, optimizer, criterion, CLIP), 5)
    valid_loss = round(validating(model, valid_iterator, criterion),5)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
    
    print('Epoch: {} \n Train Loss {}  Val loss {}:'.format(epoch, train_loss, valid_loss))
    print('Train Perplexity {}  Val Perplexity {}:'.format(np.exp(train_loss), np.exp(valid_loss)))


Epoch: 0 
 Train Loss 2.91818  Val loss 4.09754:
Train Perplexity 18.507573006029325  Val Perplexity 60.19203291789306:



Epoch: 1 
 Train Loss 2.44542  Val loss 3.94528:
Train Perplexity 11.53539344680723  Val Perplexity 51.690809512679955:



Epoch: 2 
 Train Loss 2.29778  Val loss 4.07635:
Train Perplexity 9.952064329967525  Val Perplexity 58.929982388782605:



Epoch: 3 
 Train Loss 2.16294  Val loss 4.11245:
Train Perplexity 8.696668314854502  Val Perplexity 61.0962200934771:



Epoch: 4 
 Train Loss 2.10557  Val loss 3.98642:
Train Perplexity 8.211782391808516  Val Perplexity 53.86171884095343:



Epoch: 5 
 Train Loss 2.01966  Val loss 3.91625:
Train Perplexity 7.535762338850281  Val Perplexity 50.2117970457174:



Epoch: 6 
 Train Loss 1.96633  Val loss 3.84996:
Train Perplexity 7.144408337175365  Val Perplexity 46.99118354664396:



Epoch: 7 
 Train Loss 1.88621  Val loss 3.68458:
Train Perplexity 6.594328754375952  Val Perplexity 39.8283910122803:



Epoch: 8 
 Train Loss 1.83477  Val loss 3.6394:
Train Perplexity 6.263693330282359  Val Perplexity 38.068988478523295:



Epoch: 9 
 Train Loss 1.75529  Val loss 3.5026:
Train Perplexity 5.78512518501943  Val Perplexity 33.20166416108182:


In [30]:
model.load_state_dict(torch.load("model.pt"))

<All keys matched successfully>

In [31]:
test_loss = validating(model, test_iterator, criterion)

print('| Test Loss: {} Test PPL:{}|'.format(test_loss, np.exp(test_loss)))

| Test Loss: 3.618312567472458 Test PPL:37.27461632375184|


In [32]:
def translate(sentence):
    sent_vec = SRC.process([sentence]).to(device)
    input = torch.zeros((10, 1)).type(torch.LongTensor).to(device)
    input += SRC.vocab.stoi['<sos>']
    output = model(input, input, teacher_forcing_ratio = 0) # get model output
    out = []
    for t in output:
        if t[0].max(0)[1] != SRC.vocab.stoi['<eos>']:
            out += [TRG.vocab.itos[t[0].max(0)[1]]]
        else:
            break
    return " ".join(out[::-1])

In [33]:
translate("ein klein gespenster")

'a group of the . <unk>'

In [34]:
def bleu(sentence, target):
    hyp = translate(sentence)
    return sentence_bleu([target], hyp)

In [35]:
bleu("ein klein gespenster", "a little ghost")

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.4774108847934616